<a href="https://colab.research.google.com/github/Kevdawg1/Scraping-Airbnb/blob/master/Assignment_2_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/COMP9418/assignment2/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/COMP9418/assignment2


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Necessary libraries
import numpy as np
import pandas as pd

# combinatorics
from itertools import product, combinations
# ordered dictionaries are useful for keeping ordered sets of varibles
from collections import OrderedDict as odict

# table formating for screen output
from tabulate import tabulate

# easier debugging display
pd.set_option('display.multi_sparse', False)
from pprint import pprint

In [0]:
# https://www.cs.toronto.edu/~urtasun/courses/GraphicalModels/lecture9.pdf
# Slide 24/25
import re

graph = {}

outcomeSpace = {}

# assumes that this node has not been created before
def addNode(node, states, graph):

    if node not in graph.keys():
        graph[node] = []

    # Map outcomeSpace of node
    if node not in outcomeSpace.keys():
        t = re.search('\((.+?)\)', states)
        text_string = str(t.group(1)).strip()
        text_string = text_string.replace('"','')
        all_states = text_string.split()
        print(all_states)
        space_list = []
        i = 0

        # Special case for binary states
        if "FALSE" in all_states:
            space_list.append(0)
            all_states.remove("FALSE")
            i+=1
        if "TRUE" in all_states:
            space_list.append(1)
            all_states.remove("TRUE")
            i+=1
        
        # All other cases
        for state in all_states:
            space_list.append(i)
            i+=1
        outcomeSpace[node] = space_list
# USAGE:
# node = "HISTORY" 
# states = 'states = ( "TRUE" "FALSE" );'
# addNode(node, states, graph)
        
def addEdge(full_line, data, graph, prob_tables, outcomeSpace):
    
    text3 = ''.join(full_line)
    t = re.search('\((.+?)\)', text3)

    text_string = str(t.group(1))
    text_array = text_string.split("|")
    parents  = text_array[0].split(",")
    child = text_array[1]

    for parent in parents:
        if parent in graph.keys() and child in graph.keys() and child not in graph[parent]:
            graph[parent].append(child)
    prob_tables[child] = addProbTable(data, parents, child, outcomeSpace)

def removeNode(node, graph):

    # removes edges first
    for el in graph:
        if node in graph[el]:
            graph[el].remove(node)

    # removes node from graph
    if node in graph.keys():
        del graph[node]
    return

# USAGE:
# removeNode('LVFAILURE', graph)
# graph

In [0]:
prob_tables = {}

def printFactor(f):
    """
    argument 
    `f`, a factor to print on screen
    """
    # Create a empty list that we will fill in with the probability table entries
    table = list()
    
    # Iterate over all keys and probability values in the table
    for key, item in f['table'].items():
        # Convert the tuple to a list to be able to manipulate it
        k = list(key)
        # Append the probability value to the list with key values
        k.append(item)
        # Append an entire row to the table
        table.append(k)
    # dom is used as table header. We need it converted to list
    dom = list(f['dom'])
    # Append a 'Pr' to indicate the probabity column
    dom.append('Pr')
    print(tabulate(table,headers=dom,tablefmt='orgtbl'))

def addProbTable(input_string, parents, child, outcomeSpace):
    # k = re.search('^\((.+?)$\)', input_string)
    # print(k.group(1))

    # very dirty way of extracting the probabilities
    parts = re.split('=|;',input_string)
    all_probs = parts[1].strip()[1:-1]
    probs_parts = re.split('\(|\)',all_probs)
    while("" in probs_parts) : 
        probs_parts.remove("") 
    # Result: list['0.4 0.1 0.5', '0.01 0.98 0.01']
    # print(probs_parts)
    for i in range(len(probs_parts)):
        probs_parts[i] = probs_parts[i].split()
    # Result: list[['0.4', '0.1', '0.5'], ['0.01', '0.98', '0.01']]
    # print(probs_parts)
    var_outcomes = outcomeSpace[child]
    # print(var_outcomes)

    parent_outcomes = [outcomeSpace[var] for var in (parents)]

    # print(parent_outcomes)

    all_parent_combinations = product(*parent_outcomes)
    prob_table = odict()
    for i, parent_combination in enumerate(all_parent_combinations):
        # print(parent_combination)
        cond_array = []
        j = 0
        for var_outcome in var_outcomes:
            # print(tuple(list(parent_combination)+[var_outcome]))
            # print("i:", i ,"j:", j)
            # print()
            prob_table[tuple(list(parent_combination)+[var_outcome])] = probs_parts[j][i]
            j+=1
            
    return {'dom': tuple(list(parents)+[child]), 'table': prob_table}

# USAGE:
# text4 = "data = ((0.4 0.1 0.3 0.2)(0.01 0.96 0.01 0.02)) ;"
# parents = ['HISTORY','DEAD']
# child = 'LVFAILURE'
# addProbTable(text4, parents, child, outcomeSpace)

In [30]:
f = open("inputExample.net", "r")
input_lines = f.readlines()
f.close()
for i in range(len(input_lines)):
    line = input_lines[i]
    words = line.split(" ")
    if words[0] == "node":
        print("Make new node", words[1].strip("\n"))
        addNode(words[1].strip("\n"),input_lines[i+2], graph)
        i+=3
    elif words[0] == "potential":
        print("Make new CPT", "".join(words[1:]))
        addEdge(words[1:], input_lines[i+2], graph, prob_tables, outcomeSpace)
        i+=3
graph

Make new node HISTORY
['TRUE', 'FALSE']
Make new node LVFAILURE
['TRUE', 'FALSE']
Make new node DEAD
['TRUE', 'FALSE']
Make new node ALIVE
['TRUE', 'FALSE']
Make new CPT (HISTORY,DEAD|LVFAILURE)

Make new CPT (LVFAILURE|ALIVE)



{'ALIVE': [],
 'DEAD': ['LVFAILURE'],
 'HISTORY': ['LVFAILURE'],
 'LVFAILURE': ['ALIVE']}

In [0]:
outcomeSpace

In [16]:
for elem in prob_tables:
    print(elem)
    print(printFactor(prob_tables[elem]))

LVFAILURE
|   HISTORY |   DEAD |   LVFAILURE |   Pr |
|-----------+--------+-------------+------|
|         0 |      0 |           0 | 0.4  |
|         0 |      0 |           1 | 0.01 |
|         0 |      1 |           0 | 0.1  |
|         0 |      1 |           1 | 0.96 |
|         1 |      0 |           0 | 0.3  |
|         1 |      0 |           1 | 0.01 |
|         1 |      1 |           0 | 0.2  |
|         1 |      1 |           1 | 0.02 |
None
ALIVE
|   LVFAILURE |   ALIVE |   Pr |
|-------------+---------+------|
|           0 |       0 | 0.9  |
|           0 |       1 | 0.01 |
|           1 |       0 | 0.1  |
|           1 |       1 | 0.99 |
None


In [0]:
def find_immediate_parents(G, node):
    parents = []
    for i in G:
        for j in G[i]:
            if j == node:
                parents.append(i)
    return parents

In [32]:
def pruneNode(node, evidence, graph, outcomeSpace, prob_tables):
    if len(graph[node]) == 0:
        print("leaf node")
    else:
        newOutcomeSpace = outcomeSpace
        for el in evidence:
            newOutcomeSpace[el] = [evidence[el]]
        print(newOutcomeSpace)
        var_outcomes = outcomeSpace[node]
        parents = find_immediate_parents(graph, node)
        parent_outcomes = [newOutcomeSpace[var] for var in (parents)]
        all_parent_combinations = product(*parent_outcomes)

        cpt_node = prob_tables[node]
        print(cpt_node)
        print(cpt_node['table'].keys())
        inputs_dict = {}
        inputs_arr = []
        for i, parent_combination in enumerate(all_parent_combinations):
            for var_outcome in var_outcomes:
                print(tuple(list(parent_combination)+[var_outcome]))
                inputs_arr.append(tuple(list(parent_combination)+[var_outcome]))

        cpt_values = list(cpt_node['table'].values())
        # print(cpt_values)

        prob_table = odict()
        i = 0
        for key in cpt_node['table'].keys():
            if key in inputs_arr:
                prob_table[key] = cpt_values[i]
            i+=1
        # print(prob_table)

        newGraph = graph.copy()
        newGraph[node] = []
        
        return newGraph, {'dom': tuple(list(parents)+[node]), 'table': prob_table}

            
# pruneNode("LVFAILURE", 0, graph, outcomeSpace, prob_tables)
evidence = {
    "HISTORY": 1
}
new_graph, new_table = pruneNode("LVFAILURE", evidence, graph, outcomeSpace, prob_tables)
new_table

{'HISTORY': [1], 'LVFAILURE': [0, 1], 'DEAD': [0, 1], 'ALIVE': [0, 1]}
{'dom': ('HISTORY', 'DEAD', 'LVFAILURE'), 'table': OrderedDict([((0, 0, 0), '0.4'), ((0, 0, 1), '0.01'), ((0, 1, 0), '0.1'), ((0, 1, 1), '0.96'), ((1, 0, 0), '0.3'), ((1, 0, 1), '0.01'), ((1, 1, 0), '0.2'), ((1, 1, 1), '0.02')])}
odict_keys([(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)])
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)


{'dom': ('HISTORY', 'DEAD', 'LVFAILURE'),
 'table': OrderedDict([((1, 0, 0), '0.3'),
              ((1, 0, 1), '0.01'),
              ((1, 1, 0), '0.2'),
              ((1, 1, 1), '0.02')])}

In [0]:
new_graph

In [33]:
graph

{'ALIVE': [],
 'DEAD': ['LVFAILURE'],
 'HISTORY': ['LVFAILURE'],
 'LVFAILURE': ['ALIVE']}

In [35]:
# Produce undirected graph
H = graph.copy()
for node in H:
    for child in H[node]:
        print(child,H[child])
        if node not in H[child]:
            H[child].append(node)
print(H)

for i in range(len(graph)):
    

LVFAILURE ['ALIVE', 'HISTORY', 'DEAD']
ALIVE ['LVFAILURE']
HISTORY ['LVFAILURE']
DEAD ['LVFAILURE']
LVFAILURE ['ALIVE', 'HISTORY', 'DEAD']
LVFAILURE ['ALIVE', 'HISTORY', 'DEAD']
{'HISTORY': ['LVFAILURE'], 'LVFAILURE': ['ALIVE', 'HISTORY', 'DEAD'], 'DEAD': ['LVFAILURE'], 'ALIVE': ['LVFAILURE']}
4
